In [16]:
# pip install selenium
# pip install easyocr

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import time

In [10]:
# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 드라이버 생성
driver = webdriver.Chrome(options=chrome_options)

# 브라우저 사이즈
driver.set_window_size(1900,1000)

# 웹페이지가 로드될때까지 2초를 대기
driver.implicitly_wait(time_to_wait=2)

driver.get(url='https://tickets.interpark.com/')

In [11]:
import os

# personal.txt 파일에서 아이디와 비밀번호 읽어오기
def get_credentials(file_path):
    personal = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            personal[key] = value
    return personal

# personal.txt 파일 경로
file_path = 'personal.txt'

# 아이디와 비밀번호 가져오기
personal = get_credentials(file_path)
username = personal.get('username')
password = personal.get('password')

# 가져온 아이디와 비밀번호를 사용하여 작업 수행
# print(f"Username: {username}")
# print(f"Password: {password}")

# 여기서 WebDriver 실행 등 필요한 작업을 수행합니다.


In [12]:
# 로그인
driver.find_element(By.LINK_TEXT,'로그인').click()
userId = driver.find_element(By.ID, 'userId')
userId.send_keys(username)
userPwd = driver.find_element(By.ID, "userPwd")
userPwd.send_keys(password)
userPwd.send_keys(Keys.ENTER)
# 로그인버튼 클릭
# driver.find_element(By.ID, "btn_login").click()

In [22]:
# 사이트 이동 - 현재 LCK
driver.get('https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07032&TeamCode=PE015')
# 키움 = https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07001&TeamCode=PB003
# 두산 = https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07001&TeamCode=PB004
# LCK = https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07032&TeamCode=PE015

# 첫번째 경기 예매하기 버튼 클릭 CSS_SELECTOR
driver.find_element(By.CSS_SELECTOR, f".timeScheduleList .timeSchedule:nth-of-type({1}) .BtnColor_Y.btn1").click()

In [ ]:
# import easyocr